In [1]:
from math import exp, log, sqrt

A tridiagonal solver for evolving the PDE.

In [2]:
def TridiagonalSolver(diag, upper, lower, rhs):
    size = len(rhs)
    for i in range(size-1):
        diag[i+1] = diag[i+1] - lower[i+1] / diag[i] * upper[i]
        rhs[i+1]  = rhs[i+1] - lower[i+1] / diag[i] * rhs[i]

    rhs[size-1] = rhs[size-1] / diag[size-1]
    for i in range(size-1):
        j = size - 2 - i
        rhs[j] = (rhs[j] - upper[j] * rhs[j+1]) / diag[j]

A simple test for the tridiagonal solver.
See https://matlabgeeks.weebly.com/uploads/8/0/4/8/8048228/thomas_algorithm_and_tridiagonal_matrix-_example.pdf.

In [3]:
diag = [-2.6, -2.6, -2.6, -2.6]
upper = [1., 1., 1., 1.]
lower = [1., 1., 1., 1.]
rhs = [-240., 0., 0., -150.]
TridiagonalSolver(diag, upper, lower, rhs)
print(rhs)

[118.11216764581187, 67.09163587911088, 56.32608563987643, 79.35618678456785]


In the following, we will consider several formulations of the Black-Scholes PDE with constant risk free rate, funding rate, and volatility parameters. The original PDE for option value $V(t,S)$ reads
$$\frac{\partial V}{\partial t}+r_FS\frac{\partial V}{\partial S}+\frac{1}{2}\sigma^2S^2\frac{\partial^2 V}{\partial S^2}=rV,$$
with terminal condition, $V(T,S)=g(S)$, where $g(S)$ is the terminal European payoff.

Define $x(t)=\log(S(t)/S(0))$, and let $V_1(t,x)=V(t,S(0)e^{x})$, then we have our first convenient formulation of the above PDE,
$$\frac{\partial V_1}{\partial t}+\left(r_F-\frac{1}{2}\sigma^2\right)\frac{\partial V_1}{\partial x}+\frac{1}{2}\sigma^2\frac{\partial^2 V_1}{\partial x^2}=rV_1,$$
with terminal condition, $V_1(T,x)=g(S(0)e^x)$.

Similarly, define $x(t)=\log(S(t)/F(t))$, where $F(t)=S(0)e^{r_Ft}$, and let $V_2(t,x)=V(t,F(t)e^{x})$, then we have our second convenient formulation of the above PDE,
$$\frac{\partial V_2}{\partial t}-\frac{1}{2}\sigma^2\frac{\partial V_2}{\partial x}+\frac{1}{2}\sigma^2\frac{\partial^2 V_2}{\partial x^2}=rV_2,$$
with terminal condition, $V_2(T,x)=g(F(T)e^x)$.

Now, based on the above two formulations, we can further remove the risk free rate by introducing $V_3(t,x)=e^{-rt}V(t,S(0)e^{x})$ and $V_4(t,x)=e^{-rt}V(t,F(t)e^{x})$, then the corresponding PDEs are
$$\frac{\partial V_3}{\partial t}+\left(r_F-\frac{1}{2}\sigma^2\right)\frac{\partial V_3}{\partial x}+\frac{1}{2}\sigma^2\frac{\partial^2 V_3}{\partial x^2}=0,$$
and
$$\frac{\partial V_4}{\partial t}-\frac{1}{2}\sigma^2\frac{\partial V_4}{\partial x}+\frac{1}{2}\sigma^2\frac{\partial^2 V_4}{\partial x^2}=0,$$
with terminal conditions $V_3(T,x)=e^{-rT}g(S(0)e^x)$ and $V_4(T,x)=e^{-rT}g(F(T)e^x)$, respectively.

When it comes to early exercise, if we use the explicit scheme, the free boundary condition will be approximated as
$$V_1(t,x)=\max\left(V_1(t,x), g(S(0)e^x)\right),$$
$$V_2(t,x)=\max\left(V_2(t,x), g(F(t)e^x)\right),$$
$$V_3(t,x)=\max\left(V_3(t,x), e^{-rt}g(S(0)e^x)\right),$$
$$V_4(t,x)=\max\left(V_4(t,x), e^{-rt}g(F(t)e^x)\right),$$
for the above four formulations, where the second term in each $\max$ function is the early exercise value. If we use other schemes, these will also be the early exercise value.

In the following, for the penalty method, no optimization is applied and we haved set the nonlinear iteration to a fixed number, rather than stopping the iteration with some termination condition.

In [54]:
def OneDimBackwardPDEOptionPricer(spot,
                                  repo,
                                  rate,
                                  volatility,
                                  expiry,
                                  payoff,
                                  number_of_time_steps,
                                  number_of_state_variables,
                                  number_of_standard_deviations,
                                  is_american = False,
                                  remove_funding = False,
                                  remove_discounting = False,
                                  early_exercise_policy = ["Explicit"]):
    # Calculate terms appearing in the PDE
    temp       = .5 * volatility * volatility
    convection = repo - temp
    diffusion  = temp
    source     = rate
    
    # Forward and discount curves
    # Assuming state variables x = log(S(t)/S(0)),
    # and evolving V(t,x)
    forward  = lambda t : spot
    discount = lambda t : 1.
    
    # If using x(t) = log(S(t)/F(t)) as state variables
    if remove_funding:
        convection = -temp
        forward    = lambda t : spot * exp(repo * t)
        
    # If evolving e^{-rt}V(t,x)
    if remove_discounting:
        source   = 0.
        discount = lambda t : exp(-rate * t)
        
    # Set up finite difference grid
    # Temporal direction
    # Evolving the PDE forward with negative time step ==> backward induction
    time_step = -expiry / number_of_time_steps
    times = []
    for i in range(number_of_time_steps):
        times.append(expiry - expiry * (i+1) / number_of_time_steps)
    
    # Spatial direction
    standard_deviation = volatility * sqrt(expiry)
    if number_of_state_variables % 2 == 0:
        raise ValueError("Number of state variables must be odd")
    
    central_index = number_of_state_variables // 2
    state_variable_step = number_of_standard_deviations * standard_deviation / central_index
    state_variables = [0.] * number_of_state_variables
    for i in range(number_of_state_variables):
        state_variables[i] = (i - central_index) * state_variable_step
        
    # Now, we are ready for backward induction
    # First, set up initial condition
    exp_state_variables = [exp(sv) for sv in state_variables]
    prices = [discount(expiry) * payoff(forward(expiry) * esv) for esv in exp_state_variables]
    
    # Evolve with the implicit scheme
    dt_over_dx = time_step / state_variable_step
    dt_over_dx_dx = dt_over_dx / state_variable_step
    k = number_of_state_variables-1
        
    diag  = [0.] * number_of_state_variables
    upper = [0.] * number_of_state_variables
    lower = [0.] * number_of_state_variables
    for time in times:
        for j in range(number_of_state_variables):
            diag[j]  = 1. - source * time_step - 2. * diffusion * dt_over_dx_dx;
            upper[j] =   .5 * convection * dt_over_dx + diffusion * dt_over_dx_dx;
            lower[j] = - .5 * convection * dt_over_dx + diffusion * dt_over_dx_dx;
            
        # Apply boundary conditions -- does not matter that much
        prices[0] = prices[0] - lower[0] * payoff(forward(time) * exp_state_variables[0])
        prices[k] = prices[k] - upper[k] * payoff(forward(time) * exp_state_variables[k])
        
        
        if is_american:
            exercises = [discount(time) * payoff(forward(time) * esv) for esv in exp_state_variables]
            
            if early_exercise_policy[0] == "Explicit":
                TridiagonalSolver(diag, upper, lower, prices)
                prices = [exercise if exercise > price else price for (price, exercise) in zip(prices, exercises)]
            elif early_exercise_policy[0] == "Penalty":
                large = early_exercise_policy[1]
                new_prices = [price for price in prices]
                
                for k in range(10):
                    p = [large if price < exercise else 0 for (price, exercise) in zip(new_prices, exercises)]
                    new_diag = [a + b for (a, b) in zip(diag, p)]
                    new_upper = [u for u in upper]
                    new_lower = [l for l in lower]
                    new_prices = [a + b * exercise for (a, b, exercise) in zip(prices, p, exercises)]
                    TridiagonalSolver(new_diag, new_upper, new_lower, new_prices)
                    
                prices = new_prices
            else:
                raise ValueError("Do not understand the early exercise policy")
        else:
            TridiagonalSolver(diag, upper, lower, prices)
        
    return prices[central_index]

In the following, we calculate both European and American option values using the finite difference scheme implemented above. For benchmark, we have also performed the calculation for the same options in F3, and the benchmark results are listed below.
```
|      |   European  |   American  |
|------|-------------|-------------|
| Call | 8.735090623 | 8.735074661 |
|------|-------------|-------------|
| Put  | 6.8120137   | 7.022105627 |
```

In [62]:
spot = 100.
strike = 100.
repo = .02
rate = .04
volatility = .2
expiry = 1.
number_of_time_steps = 400
number_of_state_variables = 1073
number_of_standard_deviations = 4.5

In [63]:
# Put option
diff = lambda x : strike - x
payoff = lambda x : diff(x) if diff(x) > 0. else 0.

In [64]:
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    False, 
                                    False, 
                                    False,
                                    ["Penalty", 10**6]))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    False, 
                                    False,
                                    True,
                                    ["Penalty", 10**6]))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    False, 
                                    True, 
                                    False,
                                    ["Penalty", 10**6]))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    False, 
                                    True,
                                    True,
                                    ["Penalty", 10**6]))

6.795891831599293
6.796194536198399
6.795755519617307
6.796149144298232


In [65]:
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    True, 
                                    False, 
                                    False,
                                    ["Penalty", 10**6]))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    True, 
                                    False,
                                    True,
                                    ["Penalty", 10**6]))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    True, 
                                    True, 
                                    False,
                                    ["Penalty", 10**6]))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    True, 
                                    True,
                                    True,
                                    ["Penalty", 10**6]))

7.014182529620598
7.013982697263918
7.0137115149311695
7.014117258614161


In [66]:
# Call option
diff = lambda x : x - spot
payoff = lambda x : diff(x) if diff(x) > 0. else 0.

In [67]:
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    False, 
                                    False, 
                                    False))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    False, 
                                    False,
                                    True))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    False, 
                                    True, 
                                    False))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    False, 
                                    True,
                                    True))

8.736668803914183
8.73718642668308
8.736678681713235
8.737069452987134


In [68]:
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    True, 
                                    False, 
                                    False,
                                    ["Penalty", 10**6]))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    True, 
                                    False,
                                    True,
                                    ["Penalty", 10**6]))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    True, 
                                    True, 
                                    False,
                                    ["Penalty", 10**6]))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    True, 
                                    True,
                                    True,
                                    ["Penalty", 10**6]))

8.736676419888553
8.737155667389077
8.736685388054854
8.73705107409194
